# Úkoly:
1. Pomocí Numpy a Numby napište maticové násobení dvou čtvercových matic.
    - Výsledek ukládejte do předem vytvořené matice C, tedy funkce bude mít tvar `def matrix_multiplication(A, B, C)`.
    - Zakompilujte funkci a na maticích o velikosti 1000x1000 změřte čas výpočtu maticového násobení.
    - Porovnejte výsledek s výsledkem z Numpy.


In [1]:
import numpy as np
import numba
import math

In [2]:
n = 1000

A = np.random.rand(n, n)
B = np.random.rand(n, n)
C = np.zeros((n, n), dtype=np.float64)

In [3]:
def matrix_multiply(A, B, C):
    for i in range(n):
        for k in range(n):
            for j in range(n):
                C[i, j] += A[i, k] * B[k, j]

In [4]:
matrix_multiply_jitted = numba.jit(matrix_multiply, nopython=True)

In [5]:
%time matrix_multiply_jitted(A, B, C)

CPU times: user 1.21 s, sys: 20.1 ms, total: 1.23 s
Wall time: 1.02 s


In [7]:
%time np.dot(A, B, out=C)

CPU times: user 104 ms, sys: 4.25 ms, total: 109 ms
Wall time: 44.8 ms


array([[257.1423091 , 258.73470498, 252.11213296, ..., 245.44149976,
        249.15237933, 250.60057032],
       [243.85566761, 246.53479724, 252.967652  , ..., 243.3602125 ,
        248.92440703, 243.28646382],
       [249.93624277, 254.59866034, 255.47037946, ..., 246.71918673,
        261.44311705, 255.66891865],
       ...,
       [251.43595322, 257.48564482, 255.26633517, ..., 246.39751385,
        255.7882276 , 254.82178154],
       [252.04134528, 254.342169  , 250.33986999, ..., 244.59918244,
        256.96508296, 252.86870092],
       [247.76491265, 249.03669752, 250.22121626, ..., 242.54877997,
        246.1260055 , 243.83282308]], shape=(1000, 1000))

In [6]:
@numba.jit(nopython=True, parallel=True)
def numba_matrix_multiply(A, B, C):
    for i in numba.prange(n):
        for k in numba.prange(n):
            for j in numba.prange(n):
                C[i, j] += A[i, k] * B[k, j]

In [7]:
%time numba_matrix_multiply(A, B, C)

/home/fei/pah0008/VVP/.venv/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)


CPU times: user 1.06 s, sys: 4.24 ms, total: 1.06 s
Wall time: 603 ms



2. Upravte kód tak, aby výpočet probíhal v tzv. Chunk módu. To znamená, že výpočet bude rozdělen do menších bloků, které budou postupně počítány. Výsledek bude stejný jako v bodě 1, ale výpočet bude probíhat v menších blocích.
    - výsledný kód bude mít 6 vnořených for cyklů 
        - nejprve 3 přes chunky v každé z OS (i,j,k)
        - ve vnitř 3 cykly přes prvky v každém chunku
    - zakompilujte, očasujte, a porovnejte s výsledkem z bodu 1 a s výsledkem z Numpy.


In [8]:
n = 128*16

A = np.random.rand(n, n)
B = np.random.rand(n, n)
C_numba = np.zeros((n, n), dtype=np.float64)

In [33]:
def chunk_matrix_multiply(A, B, C):
    chunk_size = 128
    n_chunks = A.shape[0] // chunk_size
    for i_chunk in numba.prange(n_chunks):
        for k_chunk in numba.prange(n_chunks):
            for j_chunk in numba.prange(n_chunks):
                for i in range(i_chunk*chunk_size, (i_chunk + 1)*chunk_size):
                    for k in range(k_chunk*chunk_size, (k_chunk + 1)*chunk_size):
                        for j in range(j_chunk*chunk_size, (j_chunk + 1)*chunk_size):
                            C[i, j] += A[i, k] * B[k, j]

In [35]:
chunk_matrix_multiply_jitted = numba.jit(chunk_matrix_multiply, nopython=True, parallel=True)
%time chunk_matrix_multiply_jitted(A, B, C_numba)

CPU times: user 8.99 s, sys: 7.92 ms, total: 9 s
Wall time: 2.89 s


In [24]:
%time numba_matrix_multiply(A, B, C_numba)

CPU times: user 805 ms, sys: 3.86 ms, total: 809 ms
Wall time: 314 ms



3. Upravte funkci tak ať výpočet probíhá paralelně, otestujte jaký vliv má když jednotlivé z vnějších smyček nastavíte jako paralelní.
    - opět porovnejte výsledky s výsledkem z Numpy a s výsledkem z bodu 1 a 2.